# Power Predict 

The first step is to upload the data and pre-process it.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os 
from google.cloud import bigquery
import pandas_gbq
## from colorama import Fore, Style
from pathlib import Path
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split

## 1. We import the first file of the project 'Electricity_Data_Explorer.csv'

In [2]:
electricity_data_explorer = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/raw_data/Electricity_Data_Explorer.csv')

In [3]:
electricity_data_explorer

,Country,Time,Balance,Product,Value,Unit
0,Australia,July 2023,Net Electricity Production,Electricity,22460.2248,GWh
1,Australia,July 2023,Net Electricity Production,Total Combustible Fuels,15153.4307,GWh
2,Australia,July 2023,Net Electricity Production,"Coal, Peat and Manufactured Gases",10684.9635,GWh
3,Australia,July 2023,Net Electricity Production,Oil and Petroleum Products,360.4031,GWh
4,Australia,July 2023,Net Electricity Production,Natural Gas,3848.6375,GWh
...,...,...,...,...,...,...
134466,IEA Total,January 2010,Total Imports,Electricity,34846.5580,GWh
134467,IEA Total,January 2010,Total Exports,Electricity,32372.5830,GWh
134468,IEA Total,January 2010,Used for pumped storage,Electricity,6869.2530,GWh
134469,IEA Total,January 2010,Distribution Losses,Electricity,62000.7330,GWh


In [4]:
products = electricity_data_explorer['Product'].unique()
##products

### We create a list to be used as a filter on specific countries:

In [5]:
##filter_countries = ['Australia', 'Germany']

##['Germany', 'Argentina', 'Mexico', 'Canada', 'Spain', 'Norway', 'England', 'Tunisia', 'Morocco', 'Nigeria', 
            ## 'South Africa', 'Angola', 'Turkey', 'Iran', 'Australia', 'Thailand', 'Japan', 'South Korea', 'Liberia', 'France']

### We create a list to be used as a filter on the outputs:

In [6]:
filter_balance = ['Net Electricity Production']
                  ##, 'Final Consumption (Calculated)']

### We create a list to be used as a filter on the different sources of energy:

In [7]:
filter_products = ['Combustible Renewables', 'Hydro', 'Wind', 'Solar',
                   'Total Renewables (Hydro, Geo, Solar, Wind, Other)', 'Other Renewables']

### We apply the filters to the initial dataframe:

In [8]:
electricity_data_explorer = electricity_data_explorer[electricity_data_explorer['Product'].isin(filter_products)]
## electricity_data_explorer

In [9]:
##electricity_data_explorer = electricity_data_explorer[electricity_data_explorer['Country'].isin(filter_countries)]

In [10]:
electricity_data_explorer = electricity_data_explorer[electricity_data_explorer['Balance'].isin(filter_balance)]
##electricity_data_explorer

### Reordering the columns

In [11]:
electricity_data_explorer = electricity_data_explorer[['Time', 'Country', 'Balance','Product','Value']]
#electricity_data_explorer

### Reindexing the dataframe

In [12]:
electricity_data_explorer.index = range(len(electricity_data_explorer.index))
#electricity_data_explorer

In [13]:
electricity_data_explorer = electricity_data_explorer.pivot_table('Value', ['Time','Country', 'Balance'], 'Product')

In [14]:
electricity_data_explorer = electricity_data_explorer.fillna(0)
#electricity_data_explorer

In [15]:
electricity_data_explorer = electricity_data_explorer.reset_index()
#electricity_data_explorer

In [16]:
electricity_data_explorer['Time'].min()

'April 2010'

In [17]:
specific_country = 'Canada'
filtered_df = electricity_data_explorer[electricity_data_explorer['Country'] == specific_country]


In [18]:
df = electricity_data_explorer

features = ['Combustible Renewables', 'Hydro', 'Wind', 'Solar',
            'Total Renewables (Hydro, Geo, Solar, Wind, Other)', 'Other Renewables']

##df['Time'] = df.index.month
combustible_renewables = df[['Time', 'Country','Combustible Renewables']]
hydro = df[['Time', 'Country','Hydro']]
wind= df[['Time', 'Country','Wind']]
solar = df[['Time', 'Country','Solar']]
total_renewables = df[['Time', 'Country','Total Renewables (Hydro, Geo, Solar, Wind, Other)']]
other_renewables = df[['Time', 'Country','Other Renewables']]

## 2. Back-up file if missing data 'weather_data_temperature_radiation.csv' (we don't need it anymore!)

In [19]:
temp_radiation = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/raw_data/weather_data_temperature_radiation.csv')
temp_radiation = temp_radiation.loc[(temp_radiation['utc_timestamp'] >= '2010-01-01')] ## Data from 2010 on only
temp_radiation = temp_radiation.reset_index()
temp_radiation = temp_radiation.drop(['index'], axis=1)
##temp_radiation

In [20]:
temp_radiation_transposed = temp_radiation.transpose()
temp_radiation_transposed = temp_radiation_transposed.reset_index()
##temp_radiation_transposed

In [21]:
temp_radiation_transposed.rename({'index': 'prepar'}, inplace=True, axis=1)
##temp_radiation_transposed.head()

In [22]:
clean1 = temp_radiation_transposed['prepar'].unique()
##clean1

## 3. We import and clean all the files containing weather data

In [23]:
CDD_18 = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/raw_data/CDD_18.csv')
CDD_21 = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/raw_data/CDD_21.csv')
Global_Horizontal_Irrandiance = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/raw_data/Global_Horizontal_Irrandiance.csv')
HDD_16 = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/raw_data/HDD_16.csv')
HDD_18 = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/raw_data/HDD_18.csv')
Heat_index = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/raw_data/Heat_index.csv')
Relative_Humidty = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/raw_data/Relative_Humidty.csv')
Temperature = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/raw_data/Temperature.csv')
Total_Precipitation = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/raw_data/Total_Precipitation.csv')

In [24]:
#CDD_21

In [25]:
##list(CDD_18.columns)

In [26]:
## Converting the date columns into rows
CDD_18=pd.melt(CDD_18, id_vars=['Country','parameter'],value_vars=['Jan-10',
 'Feb-10','Mar-10','Apr-10','May-10','Jun-10','Jul-10', 'Aug-10','Sep-10','Oct-10', 'Nov-10', 'Dec-10','Jan-11', 'Feb-11',
 'Mar-11', 'Apr-11', 'May-11', 'Jun-11', 'Jul-11', 'Aug-11', 'Sep-11', 'Oct-11', 'Nov-11', 'Dec-11', 'Jan-12', 'Feb-12', 'Mar-12', 'Apr-12',
 'May-12','Jun-12','Jul-12','Aug-12', 'Sep-12','Oct-12','Nov-12', 'Dec-12','Jan-13',
 'Feb-13','Mar-13','Apr-13', 'May-13', 'Jun-13', 'Jul-13', 'Aug-13', 'Sep-13', 'Oct-13', 'Nov-13', 'Dec-13', 'Jan-14', 'Feb-14', 'Mar-14',
 'Apr-14', 'May-14', 'Jun-14', 'Jul-14', 'Aug-14', 'Sep-14', 'Oct-14', 'Nov-14', 'Dec-14', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15', 
'Jul-15','Aug-15','Sep-15', 'Oct-15', 'Nov-15', 'Dec-15', 'Jan-16', 'Feb-16', 'Mar-16','Apr-16', 'May-16', 'Jun-16', 'Jul-16', 'Aug-16', 'Sep-16',
 'Oct-16','Nov-16','Dec-16','Jan-17','Feb-17','Mar-17','Apr-17','May-17','Jun-17','Jul-17','Aug-17','Sep-17','Oct-17','Nov-17','Dec-17','Jan-18','Feb-18','Mar-18', 'Apr-18',
 'May-18','Jun-18', 'Jul-18', 'Aug-18','Sep-18', 'Oct-18', 'Nov-18', 'Dec-18', 'Jan-19', 'Feb-19', 'Mar-19', 'Apr-19', 'May-19', 'Jun-19', 'Jul-19', 'Aug-19', 'Sep-19',
 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20','Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20', 'Sep-20', 'Oct-20', 'Nov-20', 'Dec-20',
 'Jan-21', 'Feb-21', 'Mar-21', 'Apr-21', 'May-21', 'Jun-21', 'Jul-21','Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22',
 'Apr-22','May-22', 'Jun-22', 'Jul-22', 'Aug-22', 'Sep-22', 'Oct-22', 'Nov-22', 'Dec-22', 'Jan-23', 'Feb-23', 'Mar-23', 'Apr-23',
 'May-23', 'Jun-23', 'Jul-23', 'Aug-23','Sep-23'],var_name='Month_year',value_name='value_CDD_18')
#CDD_18

In [27]:
## Converting the date columns into rows
CDD_21=pd.melt(CDD_21, id_vars=['Country','parameter'],value_vars=['Jan-10',
 'Feb-10','Mar-10','Apr-10','May-10','Jun-10','Jul-10', 'Aug-10','Sep-10','Oct-10', 'Nov-10', 'Dec-10','Jan-11', 'Feb-11',
 'Mar-11', 'Apr-11', 'May-11', 'Jun-11', 'Jul-11', 'Aug-11', 'Sep-11', 'Oct-11', 'Nov-11', 'Dec-11', 'Jan-12', 'Feb-12', 'Mar-12', 'Apr-12',
 'May-12','Jun-12','Jul-12','Aug-12', 'Sep-12','Oct-12','Nov-12', 'Dec-12','Jan-13',
 'Feb-13','Mar-13','Apr-13', 'May-13', 'Jun-13', 'Jul-13', 'Aug-13', 'Sep-13', 'Oct-13', 'Nov-13', 'Dec-13', 'Jan-14', 'Feb-14', 'Mar-14',
 'Apr-14', 'May-14', 'Jun-14', 'Jul-14', 'Aug-14', 'Sep-14', 'Oct-14', 'Nov-14', 'Dec-14', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15', 
'Jul-15','Aug-15','Sep-15', 'Oct-15', 'Nov-15', 'Dec-15', 'Jan-16', 'Feb-16', 'Mar-16','Apr-16', 'May-16', 'Jun-16', 'Jul-16', 'Aug-16', 'Sep-16',
 'Oct-16','Nov-16','Dec-16','Jan-17','Feb-17','Mar-17','Apr-17','May-17','Jun-17','Jul-17','Aug-17','Sep-17','Oct-17','Nov-17','Dec-17','Jan-18','Feb-18','Mar-18', 'Apr-18',
 'May-18','Jun-18', 'Jul-18', 'Aug-18','Sep-18', 'Oct-18', 'Nov-18', 'Dec-18', 'Jan-19', 'Feb-19', 'Mar-19', 'Apr-19', 'May-19', 'Jun-19', 'Jul-19', 'Aug-19', 'Sep-19',
 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20','Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20', 'Sep-20', 'Oct-20', 'Nov-20', 'Dec-20',
 'Jan-21', 'Feb-21', 'Mar-21', 'Apr-21', 'May-21', 'Jun-21', 'Jul-21','Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22',
 'Apr-22','May-22', 'Jun-22', 'Jul-22', 'Aug-22', 'Sep-22', 'Oct-22', 'Nov-22', 'Dec-22', 'Jan-23', 'Feb-23', 'Mar-23', 'Apr-23',
 'May-23', 'Jun-23', 'Jul-23', 'Aug-23','Sep-23'],var_name='Month_year',value_name='value_CDD_21')
#CDD_21

In [28]:
## Converting the date columns into rows
Global_Horizontal_Irrandiance=pd.melt(Global_Horizontal_Irrandiance, id_vars=['Country','parameter'],value_vars=['Jan-10',
 'Feb-10','Mar-10','Apr-10','May-10','Jun-10','Jul-10', 'Aug-10','Sep-10','Oct-10', 'Nov-10', 'Dec-10','Jan-11', 'Feb-11',
 'Mar-11', 'Apr-11', 'May-11', 'Jun-11', 'Jul-11', 'Aug-11', 'Sep-11', 'Oct-11', 'Nov-11', 'Dec-11', 'Jan-12', 'Feb-12', 'Mar-12', 'Apr-12',
 'May-12','Jun-12','Jul-12','Aug-12', 'Sep-12','Oct-12','Nov-12', 'Dec-12','Jan-13',
 'Feb-13','Mar-13','Apr-13', 'May-13', 'Jun-13', 'Jul-13', 'Aug-13', 'Sep-13', 'Oct-13', 'Nov-13', 'Dec-13', 'Jan-14', 'Feb-14', 'Mar-14',
 'Apr-14', 'May-14', 'Jun-14', 'Jul-14', 'Aug-14', 'Sep-14', 'Oct-14', 'Nov-14', 'Dec-14', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15', 
'Jul-15','Aug-15','Sep-15', 'Oct-15', 'Nov-15', 'Dec-15', 'Jan-16', 'Feb-16', 'Mar-16','Apr-16', 'May-16', 'Jun-16', 'Jul-16', 'Aug-16', 'Sep-16',
 'Oct-16','Nov-16','Dec-16','Jan-17','Feb-17','Mar-17','Apr-17','May-17','Jun-17','Jul-17','Aug-17','Sep-17','Oct-17','Nov-17','Dec-17','Jan-18','Feb-18','Mar-18', 'Apr-18',
 'May-18','Jun-18', 'Jul-18', 'Aug-18','Sep-18', 'Oct-18', 'Nov-18', 'Dec-18', 'Jan-19', 'Feb-19', 'Mar-19', 'Apr-19', 'May-19', 'Jun-19', 'Jul-19', 'Aug-19', 'Sep-19',
 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20','Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20', 'Sep-20', 'Oct-20', 'Nov-20', 'Dec-20',
 'Jan-21', 'Feb-21', 'Mar-21', 'Apr-21', 'May-21', 'Jun-21', 'Jul-21','Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22',
 'Apr-22','May-22', 'Jun-22', 'Jul-22', 'Aug-22', 'Sep-22', 'Oct-22', 'Nov-22', 'Dec-22', 'Jan-23', 'Feb-23', 'Mar-23', 'Apr-23',
 'May-23', 'Jun-23', 'Jul-23', 'Aug-23','Sep-23'],var_name='Month_year',value_name='value_Global_Horizontal_Irrandiance')
#Global_Horizontal_Irrandiance

In [29]:
## Converting the date columns into rows
HDD_16=pd.melt(HDD_16, id_vars=['Country','parameter'],value_vars=['Jan-10',
 'Feb-10','Mar-10','Apr-10','May-10','Jun-10','Jul-10', 'Aug-10','Sep-10','Oct-10', 'Nov-10', 'Dec-10','Jan-11', 'Feb-11',
 'Mar-11', 'Apr-11', 'May-11', 'Jun-11', 'Jul-11', 'Aug-11', 'Sep-11', 'Oct-11', 'Nov-11', 'Dec-11', 'Jan-12', 'Feb-12', 'Mar-12', 'Apr-12',
 'May-12','Jun-12','Jul-12','Aug-12', 'Sep-12','Oct-12','Nov-12', 'Dec-12','Jan-13',
 'Feb-13','Mar-13','Apr-13', 'May-13', 'Jun-13', 'Jul-13', 'Aug-13', 'Sep-13', 'Oct-13', 'Nov-13', 'Dec-13', 'Jan-14', 'Feb-14', 'Mar-14',
 'Apr-14', 'May-14', 'Jun-14', 'Jul-14', 'Aug-14', 'Sep-14', 'Oct-14', 'Nov-14', 'Dec-14', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15', 
'Jul-15','Aug-15','Sep-15', 'Oct-15', 'Nov-15', 'Dec-15', 'Jan-16', 'Feb-16', 'Mar-16','Apr-16', 'May-16', 'Jun-16', 'Jul-16', 'Aug-16', 'Sep-16',
 'Oct-16','Nov-16','Dec-16','Jan-17','Feb-17','Mar-17','Apr-17','May-17','Jun-17','Jul-17','Aug-17','Sep-17','Oct-17','Nov-17','Dec-17','Jan-18','Feb-18','Mar-18', 'Apr-18',
 'May-18','Jun-18', 'Jul-18', 'Aug-18','Sep-18', 'Oct-18', 'Nov-18', 'Dec-18', 'Jan-19', 'Feb-19', 'Mar-19', 'Apr-19', 'May-19', 'Jun-19', 'Jul-19', 'Aug-19', 'Sep-19',
 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20','Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20', 'Sep-20', 'Oct-20', 'Nov-20', 'Dec-20',
 'Jan-21', 'Feb-21', 'Mar-21', 'Apr-21', 'May-21', 'Jun-21', 'Jul-21','Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22',
 'Apr-22','May-22', 'Jun-22', 'Jul-22', 'Aug-22', 'Sep-22', 'Oct-22', 'Nov-22', 'Dec-22', 'Jan-23', 'Feb-23', 'Mar-23', 'Apr-23',
 'May-23', 'Jun-23', 'Jul-23', 'Aug-23','Sep-23'],var_name='Month_year',value_name='value_HDD_16')
#HDD_16

In [30]:
## Converting the date columns into rows
HDD_18=pd.melt(HDD_18, id_vars=['Country','parameter'],value_vars=['Jan-10',
 'Feb-10','Mar-10','Apr-10','May-10','Jun-10','Jul-10', 'Aug-10','Sep-10','Oct-10', 'Nov-10', 'Dec-10','Jan-11', 'Feb-11',
 'Mar-11', 'Apr-11', 'May-11', 'Jun-11', 'Jul-11', 'Aug-11', 'Sep-11', 'Oct-11', 'Nov-11', 'Dec-11', 'Jan-12', 'Feb-12', 'Mar-12', 'Apr-12',
 'May-12','Jun-12','Jul-12','Aug-12', 'Sep-12','Oct-12','Nov-12', 'Dec-12','Jan-13',
 'Feb-13','Mar-13','Apr-13', 'May-13', 'Jun-13', 'Jul-13', 'Aug-13', 'Sep-13', 'Oct-13', 'Nov-13', 'Dec-13', 'Jan-14', 'Feb-14', 'Mar-14',
 'Apr-14', 'May-14', 'Jun-14', 'Jul-14', 'Aug-14', 'Sep-14', 'Oct-14', 'Nov-14', 'Dec-14', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15', 
'Jul-15','Aug-15','Sep-15', 'Oct-15', 'Nov-15', 'Dec-15', 'Jan-16', 'Feb-16', 'Mar-16','Apr-16', 'May-16', 'Jun-16', 'Jul-16', 'Aug-16', 'Sep-16',
 'Oct-16','Nov-16','Dec-16','Jan-17','Feb-17','Mar-17','Apr-17','May-17','Jun-17','Jul-17','Aug-17','Sep-17','Oct-17','Nov-17','Dec-17','Jan-18','Feb-18','Mar-18', 'Apr-18',
 'May-18','Jun-18', 'Jul-18', 'Aug-18','Sep-18', 'Oct-18', 'Nov-18', 'Dec-18', 'Jan-19', 'Feb-19', 'Mar-19', 'Apr-19', 'May-19', 'Jun-19', 'Jul-19', 'Aug-19', 'Sep-19',
 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20','Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20', 'Sep-20', 'Oct-20', 'Nov-20', 'Dec-20',
 'Jan-21', 'Feb-21', 'Mar-21', 'Apr-21', 'May-21', 'Jun-21', 'Jul-21','Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22',
 'Apr-22','May-22', 'Jun-22', 'Jul-22', 'Aug-22', 'Sep-22', 'Oct-22', 'Nov-22', 'Dec-22', 'Jan-23', 'Feb-23', 'Mar-23', 'Apr-23',
 'May-23', 'Jun-23', 'Jul-23', 'Aug-23','Sep-23'],var_name='Month_year',value_name='value_HDD_18')
#HDD_18

In [31]:
## Converting the date columns into rows
Heat_index=pd.melt(Heat_index, id_vars=['Country','parameter'],value_vars=['Jan-10',
 'Feb-10','Mar-10','Apr-10','May-10','Jun-10','Jul-10', 'Aug-10','Sep-10','Oct-10', 'Nov-10', 'Dec-10','Jan-11', 'Feb-11',
 'Mar-11', 'Apr-11', 'May-11', 'Jun-11', 'Jul-11', 'Aug-11', 'Sep-11', 'Oct-11', 'Nov-11', 'Dec-11', 'Jan-12', 'Feb-12', 'Mar-12', 'Apr-12',
 'May-12','Jun-12','Jul-12','Aug-12', 'Sep-12','Oct-12','Nov-12', 'Dec-12','Jan-13',
 'Feb-13','Mar-13','Apr-13', 'May-13', 'Jun-13', 'Jul-13', 'Aug-13', 'Sep-13', 'Oct-13', 'Nov-13', 'Dec-13', 'Jan-14', 'Feb-14', 'Mar-14',
 'Apr-14', 'May-14', 'Jun-14', 'Jul-14', 'Aug-14', 'Sep-14', 'Oct-14', 'Nov-14', 'Dec-14', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15', 
'Jul-15','Aug-15','Sep-15', 'Oct-15', 'Nov-15', 'Dec-15', 'Jan-16', 'Feb-16', 'Mar-16','Apr-16', 'May-16', 'Jun-16', 'Jul-16', 'Aug-16', 'Sep-16',
 'Oct-16','Nov-16','Dec-16','Jan-17','Feb-17','Mar-17','Apr-17','May-17','Jun-17','Jul-17','Aug-17','Sep-17','Oct-17','Nov-17','Dec-17','Jan-18','Feb-18','Mar-18', 'Apr-18',
 'May-18','Jun-18', 'Jul-18', 'Aug-18','Sep-18', 'Oct-18', 'Nov-18', 'Dec-18', 'Jan-19', 'Feb-19', 'Mar-19', 'Apr-19', 'May-19', 'Jun-19', 'Jul-19', 'Aug-19', 'Sep-19',
 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20','Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20', 'Sep-20', 'Oct-20', 'Nov-20', 'Dec-20',
 'Jan-21', 'Feb-21', 'Mar-21', 'Apr-21', 'May-21', 'Jun-21', 'Jul-21','Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22',
 'Apr-22','May-22', 'Jun-22', 'Jul-22', 'Aug-22', 'Sep-22', 'Oct-22', 'Nov-22', 'Dec-22', 'Jan-23', 'Feb-23', 'Mar-23', 'Apr-23',
 'May-23', 'Jun-23', 'Jul-23', 'Aug-23','Sep-23'],var_name='Month_year',value_name='value_Heat_index')
#Heat_index

In [32]:
## Converting the date columns into rows
Relative_Humidty=pd.melt(Relative_Humidty, id_vars=['Country','parameter'],value_vars=['Jan-10',
 'Feb-10','Mar-10','Apr-10','May-10','Jun-10','Jul-10', 'Aug-10','Sep-10','Oct-10', 'Nov-10', 'Dec-10','Jan-11', 'Feb-11',
 'Mar-11', 'Apr-11', 'May-11', 'Jun-11', 'Jul-11', 'Aug-11', 'Sep-11', 'Oct-11', 'Nov-11', 'Dec-11', 'Jan-12', 'Feb-12', 'Mar-12', 'Apr-12',
 'May-12','Jun-12','Jul-12','Aug-12', 'Sep-12','Oct-12','Nov-12', 'Dec-12','Jan-13',
 'Feb-13','Mar-13','Apr-13', 'May-13', 'Jun-13', 'Jul-13', 'Aug-13', 'Sep-13', 'Oct-13', 'Nov-13', 'Dec-13', 'Jan-14', 'Feb-14', 'Mar-14',
 'Apr-14', 'May-14', 'Jun-14', 'Jul-14', 'Aug-14', 'Sep-14', 'Oct-14', 'Nov-14', 'Dec-14', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15', 
'Jul-15','Aug-15','Sep-15', 'Oct-15', 'Nov-15', 'Dec-15', 'Jan-16', 'Feb-16', 'Mar-16','Apr-16', 'May-16', 'Jun-16', 'Jul-16', 'Aug-16', 'Sep-16',
 'Oct-16','Nov-16','Dec-16','Jan-17','Feb-17','Mar-17','Apr-17','May-17','Jun-17','Jul-17','Aug-17','Sep-17','Oct-17','Nov-17','Dec-17','Jan-18','Feb-18','Mar-18', 'Apr-18',
 'May-18','Jun-18', 'Jul-18', 'Aug-18','Sep-18', 'Oct-18', 'Nov-18', 'Dec-18', 'Jan-19', 'Feb-19', 'Mar-19', 'Apr-19', 'May-19', 'Jun-19', 'Jul-19', 'Aug-19', 'Sep-19',
 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20','Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20', 'Sep-20', 'Oct-20', 'Nov-20', 'Dec-20',
 'Jan-21', 'Feb-21', 'Mar-21', 'Apr-21', 'May-21', 'Jun-21', 'Jul-21','Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22',
 'Apr-22','May-22', 'Jun-22', 'Jul-22', 'Aug-22', 'Sep-22', 'Oct-22', 'Nov-22', 'Dec-22', 'Jan-23', 'Feb-23', 'Mar-23', 'Apr-23',
 'May-23', 'Jun-23', 'Jul-23', 'Aug-23','Sep-23'],var_name='Month_year',value_name='value_Relative_Humidty')
#Relative_Humidty

In [33]:
## Converting the date columns into rows
Temperature=pd.melt(Temperature, id_vars=['Country','parameter'],value_vars=['Jan-10',
 'Feb-10','Mar-10','Apr-10','May-10','Jun-10','Jul-10', 'Aug-10','Sep-10','Oct-10', 'Nov-10', 'Dec-10','Jan-11', 'Feb-11',
 'Mar-11', 'Apr-11', 'May-11', 'Jun-11', 'Jul-11', 'Aug-11', 'Sep-11', 'Oct-11', 'Nov-11', 'Dec-11', 'Jan-12', 'Feb-12', 'Mar-12', 'Apr-12',
 'May-12','Jun-12','Jul-12','Aug-12', 'Sep-12','Oct-12','Nov-12', 'Dec-12','Jan-13',
 'Feb-13','Mar-13','Apr-13', 'May-13', 'Jun-13', 'Jul-13', 'Aug-13', 'Sep-13', 'Oct-13', 'Nov-13', 'Dec-13', 'Jan-14', 'Feb-14', 'Mar-14',
 'Apr-14', 'May-14', 'Jun-14', 'Jul-14', 'Aug-14', 'Sep-14', 'Oct-14', 'Nov-14', 'Dec-14', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15', 
'Jul-15','Aug-15','Sep-15', 'Oct-15', 'Nov-15', 'Dec-15', 'Jan-16', 'Feb-16', 'Mar-16','Apr-16', 'May-16', 'Jun-16', 'Jul-16', 'Aug-16', 'Sep-16',
 'Oct-16','Nov-16','Dec-16','Jan-17','Feb-17','Mar-17','Apr-17','May-17','Jun-17','Jul-17','Aug-17','Sep-17','Oct-17','Nov-17','Dec-17','Jan-18','Feb-18','Mar-18', 'Apr-18',
 'May-18','Jun-18', 'Jul-18', 'Aug-18','Sep-18', 'Oct-18', 'Nov-18', 'Dec-18', 'Jan-19', 'Feb-19', 'Mar-19', 'Apr-19', 'May-19', 'Jun-19', 'Jul-19', 'Aug-19', 'Sep-19',
 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20','Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20', 'Sep-20', 'Oct-20', 'Nov-20', 'Dec-20',
 'Jan-21', 'Feb-21', 'Mar-21', 'Apr-21', 'May-21', 'Jun-21', 'Jul-21','Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22',
 'Apr-22','May-22', 'Jun-22', 'Jul-22', 'Aug-22', 'Sep-22', 'Oct-22', 'Nov-22', 'Dec-22', 'Jan-23', 'Feb-23', 'Mar-23', 'Apr-23',
 'May-23', 'Jun-23', 'Jul-23', 'Aug-23','Sep-23'],var_name='Month_year',value_name='value_Temperature')
#Temperature

In [34]:
## Converting the date columns into rows
Total_Precipitation=pd.melt(Total_Precipitation, id_vars=['Country','parameter'],value_vars=['Jan-10',
 'Feb-10','Mar-10','Apr-10','May-10','Jun-10','Jul-10', 'Aug-10','Sep-10','Oct-10', 'Nov-10', 'Dec-10','Jan-11', 'Feb-11',
 'Mar-11', 'Apr-11', 'May-11', 'Jun-11', 'Jul-11', 'Aug-11', 'Sep-11', 'Oct-11', 'Nov-11', 'Dec-11', 'Jan-12', 'Feb-12', 'Mar-12', 'Apr-12',
 'May-12','Jun-12','Jul-12','Aug-12', 'Sep-12','Oct-12','Nov-12', 'Dec-12','Jan-13',
 'Feb-13','Mar-13','Apr-13', 'May-13', 'Jun-13', 'Jul-13', 'Aug-13', 'Sep-13', 'Oct-13', 'Nov-13', 'Dec-13', 'Jan-14', 'Feb-14', 'Mar-14',
 'Apr-14', 'May-14', 'Jun-14', 'Jul-14', 'Aug-14', 'Sep-14', 'Oct-14', 'Nov-14', 'Dec-14', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15', 
'Jul-15','Aug-15','Sep-15', 'Oct-15', 'Nov-15', 'Dec-15', 'Jan-16', 'Feb-16', 'Mar-16','Apr-16', 'May-16', 'Jun-16', 'Jul-16', 'Aug-16', 'Sep-16',
 'Oct-16','Nov-16','Dec-16','Jan-17','Feb-17','Mar-17','Apr-17','May-17','Jun-17','Jul-17','Aug-17','Sep-17','Oct-17','Nov-17','Dec-17','Jan-18','Feb-18','Mar-18', 'Apr-18',
 'May-18','Jun-18', 'Jul-18', 'Aug-18','Sep-18', 'Oct-18', 'Nov-18', 'Dec-18', 'Jan-19', 'Feb-19', 'Mar-19', 'Apr-19', 'May-19', 'Jun-19', 'Jul-19', 'Aug-19', 'Sep-19',
 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20','Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20', 'Sep-20', 'Oct-20', 'Nov-20', 'Dec-20',
 'Jan-21', 'Feb-21', 'Mar-21', 'Apr-21', 'May-21', 'Jun-21', 'Jul-21','Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22',
 'Apr-22','May-22', 'Jun-22', 'Jul-22', 'Aug-22', 'Sep-22', 'Oct-22', 'Nov-22', 'Dec-22', 'Jan-23', 'Feb-23', 'Mar-23', 'Apr-23',
 'May-23', 'Jun-23', 'Jul-23', 'Aug-23','Sep-23'],var_name='Month_year',value_name='value_Total_Precipitation')
#Total_Precipitation

In [35]:
## Merging CDD_18 and CDD_21
concat1 = CDD_18.merge(CDD_21[['Country','Month_year', 'value_CDD_21']])
concat1 = concat1.drop(['parameter'], axis=1)

In [36]:
## Merging concat1 and Global_Horizontal_Irrandiance
concat2 = concat1.merge(Global_Horizontal_Irrandiance[['Country','Month_year', 'value_Global_Horizontal_Irrandiance']])

In [37]:
## Merging concat2 and HDD_16
concat3 = concat2.merge(HDD_16[['Country','Month_year', 'value_HDD_16']])

In [38]:
## Merging concat3 and HDD_18
concat4 = concat3.merge(HDD_18[['Country','Month_year', 'value_HDD_18']])

In [39]:
## Merging concat4 and Heat_index
concat5 = concat4.merge(Heat_index[['Country','Month_year', 'value_Heat_index']])

In [40]:
## Merging concat5 and Relative_Humidty
concat6 = concat5.merge(Relative_Humidty[['Country','Month_year', 'value_Relative_Humidty']])

In [41]:
## Merging concat6 and Temperature
concat7 = concat6.merge(Temperature[['Country','Month_year', 'value_Temperature']])

In [42]:
## Merging concat7 and Total_Precipitation
concat8 = concat7.merge(Total_Precipitation[['Country','Month_year', 'value_Total_Precipitation']])


In [43]:
## Changing the date format of the column 'Month_year' in concat8
from datetime import datetime
concat8['Month_year'] = pd.to_datetime(concat8['Month_year'], format='%b-%y')

In [44]:
## Changing the date format of the column 'Month_year' in electricity_data_explorer
electricity_data_explorer['Time'] = pd.to_datetime(electricity_data_explorer['Time'], format='%B %Y')
electricity_data_explorer = electricity_data_explorer.rename(columns={"Time": "Month_year"})
##electricity_data_explorer

In [45]:
## Merging concat8 (containing the weather data) and the electricity_data_explorer (containing the elec prod)
final = electricity_data_explorer.merge(concat8[['Country','Month_year','value_CDD_18',
 'value_CDD_21',
 'value_Global_Horizontal_Irrandiance',
 'value_HDD_16',
 'value_HDD_18',
 'value_Heat_index',
 'value_Relative_Humidty',
 'value_Temperature',
 'value_Total_Precipitation']])

In [46]:
## We add a column for the total production of wind, solar and hydro (GWh)
final['total target (wind, solar, hydro)'] = final['Wind'] + final['Solar'] + final['Hydro']

### Scaling of the columns containing the weather data

In [47]:
## We're finally going to scale the columns related to the weather data
## Linear regression model MinMaxScaler(feature_range=(0, 1)). We are dealing with GWh (energy) and prefer to have only positive values
model = MinMaxScaler(feature_range=(0, 1))

## We will only scale the columns containing 'value' in their naming
to_be_scaled = [col for col in final.columns if 'value' in col]

# Fit and transform only the selected columns
final[to_be_scaled] = model.fit_transform(final[to_be_scaled])

In [48]:
!pwd

/Users/sylvainvanhuysse/code/VonRiecken/Power-Predict/notebooks


In [49]:
os.makedirs('/Users/sylvainvanhuysse/code/VonRiecken/Power-Predict/power_predict/data', exist_ok=True)  
final.to_csv('/Users/sylvainvanhuysse/code/VonRiecken/Power-Predict/power_predict/data/merged_dataset.csv')  

# Compiling the codes into functions

## Creation of the function cleaning the energy production data

In [50]:
# We create a function with the aim of cleaning up the data related to electricity production

def clean_production_data(Electricity_Data_Explorer):

    ## We import the csv file containing the electricity production data
    ##root_path = os.path.dirname(os.path.dirname(os.path.dirname(__file__)))
    ##final_path = os.path.join(root_path, "raw_data")

    ##electricity_data_explorer = pd.read_csv(os.path.join(final_path, f'{Electricity_Data_Explorer}.csv')) ## We create a DF from the file

    ## For Jupyter Notebook we hardcode the path
    electricity_data_explorer = pd.read_csv(f'/Users/sylvainvanhuysse/code/VonRiecken/raw_data/{Electricity_Data_Explorer}.csv')
    
    ## We create a list to be used as a filter on the outputs:
    filter_balance = ['Net Electricity Production']

    ## We create a list to be used as a filter on the different sources of energy:
    filter_products = ['Combustible Renewables', 'Hydro', 'Wind', 'Solar',
                'Total Renewables (Hydro, Geo, Solar, Wind, Other)', 'Other Renewables']

    ## We apply the filters to the initial dataframe:
    electricity_data_explorer = electricity_data_explorer[electricity_data_explorer['Product'].isin(filter_products)]
    electricity_data_explorer = electricity_data_explorer[electricity_data_explorer['Balance'].isin(filter_balance)]

    ## Reordering the columns:
    electricity_data_explorer = electricity_data_explorer[['Time', 'Country', 'Balance','Product','Value']]

    ## Reindexing the dataframe:
    electricity_data_explorer.index = range(len(electricity_data_explorer.index))

    ## Pivoting the table:
    electricity_data_explorer = electricity_data_explorer.pivot_table('Value', ['Time','Country', 'Balance'], 'Product')

    ## Filling up the NaN with zeros:
    electricity_data_explorer = electricity_data_explorer.fillna(0)

    ## We reset the index:
    electricity_data_explorer = electricity_data_explorer.reset_index()

    ## Changing the date format of the column 'Month_year' in electricity_data_explorer
    electricity_data_explorer['Time'] = pd.to_datetime(electricity_data_explorer['Time'], format='%B %Y')
    electricity_data_explorer = electricity_data_explorer.rename(columns={"Time": "Month_year"})

    print("✅ Eletricity Production has been cleaned up!")

    return electricity_data_explorer ## The output of this function is a dataframe containing the electricity production data

## Creation of a function returning a dictionary with all data concerning the weather conditions

In [51]:
def storing_weather_data():

    ## We create the path to the files we want to use, placed in the folder 'raw_data'
    ##root_path = os.path.dirname(os.path.dirname(os.path.dirname(__file__)))
    ##final_path = os.path.join(root_path, "raw_data")

    ## For Jupyter Notebook we hardcode the path
    final_path = '/Users/sylvainvanhuysse/code/VonRiecken/raw_data/'

    ## We store a list of elements present in the folder
    list_dir = os.listdir(final_path)

    ## We create an empty list that will store the names of the files we want to use
    file_names = []

    ## Here we select all the csv files except Electricity_Data_Explorer that has been cleaned with another function
    for index, value in enumerate(list_dir):
        if value.endswith(".csv") and 'Electricity_Data_Explorer' not in value:
            file_names.append(value)

    ## We create a dictionary that will contain the dataframes created from each file
    dataframes = {}

    ## This for loop will store the dataframe of each csv file in a dictionary
    for file in file_names:

        name_df = file.split('.csv')[0] ## This line takes the name of the file without the extension .csv
        ## DataFrame = pd.DataFrame() ## We create a DF
        DataFrame = pd.read_csv(os.path.join(final_path, file)) ## We create a DF from the file

        ## Converting the date columns into rows
        DataFrame=pd.melt(DataFrame, id_vars=['Country','parameter'],value_vars=['Jan-10',
        'Feb-10','Mar-10','Apr-10','May-10','Jun-10','Jul-10', 'Aug-10','Sep-10','Oct-10', 'Nov-10', 'Dec-10','Jan-11', 'Feb-11',
        'Mar-11', 'Apr-11', 'May-11', 'Jun-11', 'Jul-11', 'Aug-11', 'Sep-11', 'Oct-11', 'Nov-11', 'Dec-11', 'Jan-12', 'Feb-12', 'Mar-12', 'Apr-12',
        'May-12','Jun-12','Jul-12','Aug-12', 'Sep-12','Oct-12','Nov-12', 'Dec-12','Jan-13',
        'Feb-13','Mar-13','Apr-13', 'May-13', 'Jun-13', 'Jul-13', 'Aug-13', 'Sep-13', 'Oct-13', 'Nov-13', 'Dec-13', 'Jan-14', 'Feb-14', 'Mar-14',
        'Apr-14', 'May-14', 'Jun-14', 'Jul-14', 'Aug-14', 'Sep-14', 'Oct-14', 'Nov-14', 'Dec-14', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15',
        'Jul-15','Aug-15','Sep-15', 'Oct-15', 'Nov-15', 'Dec-15', 'Jan-16', 'Feb-16', 'Mar-16','Apr-16', 'May-16', 'Jun-16', 'Jul-16', 'Aug-16', 'Sep-16',
        'Oct-16','Nov-16','Dec-16','Jan-17','Feb-17','Mar-17','Apr-17','May-17','Jun-17','Jul-17','Aug-17','Sep-17','Oct-17','Nov-17','Dec-17','Jan-18','Feb-18','Mar-18', 'Apr-18',
        'May-18','Jun-18', 'Jul-18', 'Aug-18','Sep-18', 'Oct-18', 'Nov-18', 'Dec-18', 'Jan-19', 'Feb-19', 'Mar-19', 'Apr-19', 'May-19', 'Jun-19', 'Jul-19', 'Aug-19', 'Sep-19',
        'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20','Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20', 'Sep-20', 'Oct-20', 'Nov-20', 'Dec-20',
        'Jan-21', 'Feb-21', 'Mar-21', 'Apr-21', 'May-21', 'Jun-21', 'Jul-21','Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22',
        'Apr-22','May-22', 'Jun-22', 'Jul-22', 'Aug-22', 'Sep-22', 'Oct-22', 'Nov-22', 'Dec-22', 'Jan-23', 'Feb-23', 'Mar-23', 'Apr-23',
        'May-23', 'Jun-23', 'Jul-23', 'Aug-23','Sep-23'],var_name='Month_year',value_name=f'value_{name_df}')

        dataframes[name_df] = DataFrame

    print("✅ Dictionary with weather data has been prepared!")

    return dataframes ## The output of this function is the dictionary containing all the dataframes of the weather conditions

## Creation of a function cleaning up and merging the weather data obtained with clean_production_data() and storing_weather_data()

In [52]:
def cleaning_weather_data():

    ## We call the dataframe containing the production data thanks to the function clean_production_data()
    electricity_data_explorer = clean_production_data('Electricity_Data_Explorer')

    ## We call the dictionary containing the dataframes for the weather data with the function storing_weather_data()
    dataframes = storing_weather_data()

    ## Merging CDD_18 and CDD_21
    concat1 = dataframes['CDD_18'].merge(dataframes['CDD_21'][['Country','Month_year', 'value_CDD_21']])
    concat1 = concat1.drop(['parameter'], axis=1)

    ## Merging concat1 and Global_Horizontal_Irrandiance
    concat2 = concat1.merge(dataframes['Global_Horizontal_Irrandiance'][['Country','Month_year', 'value_Global_Horizontal_Irrandiance']])

    ## Merging concat2 and HDD_16
    concat3 = concat2.merge(dataframes['HDD_16'][['Country','Month_year', 'value_HDD_16']])

    ## Merging concat3 and HDD_18
    concat4 = concat3.merge(dataframes['HDD_18'][['Country','Month_year', 'value_HDD_18']])

    ## Merging concat4 and Heat_index
    concat5 = concat4.merge(dataframes['Heat_index'][['Country','Month_year', 'value_Heat_index']])

    ## Merging concat5 and Relative_Humidty
    concat6 = concat5.merge(dataframes['Relative_Humidty'][['Country','Month_year', 'value_Relative_Humidty']])

    ## Merging concat6 and Temperature
    concat7 = concat6.merge(dataframes['Temperature'][['Country','Month_year', 'value_Temperature']])

    ## Merging concat7 and Total_Precipitation
    concat8 = concat7.merge(dataframes['Total_Precipitation'][['Country','Month_year', 'value_Total_Precipitation']])

    ## Changing the date format of the column 'Month_year' in concat8
    concat8['Month_year'] = pd.to_datetime(concat8['Month_year'], format='%b-%y')

    ## Merging concat8 (containing the weather data) and the electricity_data_explorer (containing the elec prod)
    final = electricity_data_explorer.merge(concat8[['Country','Month_year','value_CDD_18',
                                                                            'value_CDD_21',
                                                                            'value_Global_Horizontal_Irrandiance',
                                                                            'value_HDD_16',
                                                                            'value_HDD_18',
                                                                            'value_Heat_index',
                                                                            'value_Relative_Humidty',
                                                                            'value_Temperature',
                                                                            'value_Total_Precipitation']])

    ## We add a column for the total production of wind, solar, hydro (GWh)
    final['total_sol_wind_hyd'] = final['Wind'] + final['Solar'] + final['Hydro']

    # replace white spaces by underscores
    final.columns = [c.replace(' ', '_') for c in final]
    # replace "," by underscores
    final.columns = [c.replace(',', '_') for c in final]
    # replace "(" by underscores
    final.columns = [c.replace('(', '_') for c in final]
    # replace ")" by underscores
    final.columns = [c.replace(')', '_') for c in final]

    ## Specifying the data type 'datetime' of the Month_year column (needed to upload to BigQuery)
    final['Month_year'] = pd.to_datetime(final['Month_year'])

    ## We're finally going to scale the columns related to the weather data
    ## Linear regression model MinMaxScaler(feature_range=(0, 1)). We are dealing with GWh (energy) and prefer to have only positive values
    model = MinMaxScaler(feature_range=(0, 1))

    ## We will only scale the columns containing 'value' in their naming
    to_be_scaled = [col for col in final.columns if 'value' in col]
    
    # Fit and transform only the selected columns
    final[to_be_scaled] = model.fit_transform(final[to_be_scaled])

    ##os.makedirs(final_path, exist_ok=True) ## We check if the folder power_predict/data exists, if not, we create it

    root_path = root_path = os.path.dirname(os.path.dirname(os.path.dirname(__file__))) ## We call the path of the current folder
    final_path = os.path.join(root_path, "power_predict/data") ## We place ourlseves in the folder power_predict/data

    final.to_csv(f'{final_path}/merged_dataset{datetime.now()}.csv') ## We export the dataframe as csv and store it in power_predict/data and we put the timestamp at the end of the file's name

    print(f"✅ The merged_dataset{datetime.now()}.csv has been exported!")

    return final ## This function returns the final dataset with the electricity production and the weather data

## We create a function allowing us to upload the final dataframe to Big Query

In [53]:
def upload_data_bq():

    # We call the cleaned dataframe
    cleaned_weather_data = cleaning_weather_data()

    # Replace these with your own values
    project_id = 'peppy-aileron-401514'
    dataset_id = 'Power_Predict'
    table_id = 'cleaned_data'
    json_credentials_path = '/Users/sylvainvanhuysse/code/bonawa/gcp/peppy-aileron-401514-167fede484a0.json'  # Replace with the path to your service account JSON key file

    # Set up BigQuery client
    client = bigquery.Client.from_service_account_json(json_credentials_path, project=project_id)
    ##print('BQ Client is set up')

    # Set up table reference
    table_ref = client.dataset(dataset_id).table(table_id)
    ##print('BQ table reference is set up')

    # Create the schema based on DataFrame columns for the BQ table
    schema = [bigquery.SchemaField(column, cleaned_weather_data[column].dtype.name.lower()) for column in cleaned_weather_data.columns]

    # Specifying the type of the column Month_year (necessary for BigQuery)
    schema = [bigquery.SchemaField('Month_year', 'TIMESTAMP')]

    # Check if the table exists
    table_exists = False

    try:
        existing_table = client.get_table(table_ref)
        ##print(f'Table {table_ref} already exists.')
        table_exists = True
    except Exception as e:
        print('No table was found.')

    # If the table exists, delete it
    if table_exists:
        client.delete_table(table_ref)
        print(f'✅ Table {table_ref} deleted.')

    # Create the BigQuery table with the dynamically generated schema
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print(f'✅ Table {table_ref} created.')

    # Set up the destination table
    destination_table = f"{project_id}.{dataset_id}.{table_id}"

    # Upload DataFrame to BigQuery
    pandas_gbq.to_gbq(cleaned_weather_data, destination_table, project_id=project_id, if_exists='replace')

    print(f"✅ DataFrame uploaded to BigQuery table: {destination_table}")

In [54]:
## Here we call the function upload_data_bq() to upload the data in Big Query

upload_data_bq()

✅ Eletricity Production has been cleaned up!


KeyError: "The following 'id_vars' are not present in the DataFrame: ['Country', 'parameter']"

## We're importing the table created in Big Query to work on the data

In [ ]:
import pandas_gbq

# Replace these with your own values
project_id = 'peppy-aileron-401514'
dataset_id = 'Power_Predict'
table_id = 'cleaned_data'
json_credentials_path = '/Users/sylvainvanhuysse/code/bonawa/gcp/peppy-aileron-401514-167fede484a0.json'  # Replace with the path to your service account JSON key file

# Query to fetch the data (optional)
query = f'SELECT * FROM `{project_id}.{dataset_id}.{table_id}`'

# Load the DataFrame from BigQuery
cleaned_data = pandas_gbq.read_gbq(query, project_id=project_id)

cleaned_data


In [ ]:
final

In [ ]:
##final.info()

## Linear Regression

### We're here using a heatmap to see if some of our features are too correlated

In [ ]:
## We first select the columns that are gonna be part of our X 
columns_X = [col for col in final.columns if 'value' in col]

##  We're selecting the columns named with 'value'
X = final[columns_X]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute the correlation matrix
corr_matrix = X.corr()

# Create a heatmap using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:

## We create a list of the columns we want to drop because of their high correlation
columns_to_drop = ['value_CDD_18', 'value_CDD_21', 'value_HDD_16', 'value_HDD_18', 'value_Heat_index']

## We drop the columns that have a too high correlation between each other
X = X.drop(columns=columns_to_drop)

X

### Here we run our Linear Regression after having dropped the highly correlated features

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

##X = pd.get_dummies(final, columns=categorical_columns, drop_first=True)
y = final['total target (wind, solar, hydro)']

# Split training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear regression model MinMaxScaler(feature_range=(0, 1)). We are dealing with GWh (energy) and prefer to have only positive values
model = LinearRegression()

# Fit 
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
cod = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Coefficient of Determination (COD): {cod}')
print(f'R-squared: {r2}')

### Here we are feature engineering and creating a weather feature composed of the ones that are too correlated

In [ ]:
## We first select the columns that are gonna be part of our X 
columns_X = [col for col in final.columns if 'value' in col]

##  We're selecting the columns named with 'value'
X = final[columns_X]

In [ ]:
X['engineered_feature'] = X['value_CDD_18']+X['value_CDD_21']+X['value_HDD_16']+X['value_HDD_18']+X['value_Heat_index']+X['value_Temperature']
X['engineered_feature'] 

In [ ]:
## We create a list of the columns we want to drop because of their high correlation
columns_to_drop = ['value_CDD_18', 'value_CDD_21', 'value_HDD_16', 'value_HDD_18', 'value_Heat_index','value_Temperature']

## We drop the columns that have a too high correlation between each other
X = X.drop(columns=columns_to_drop)

X

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

##X = pd.get_dummies(final, columns=categorical_columns, drop_first=True)
y = final['total target (wind, solar, hydro)']

# Split training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear regression model MinMaxScaler(feature_range=(0, 1)). We are dealing with GWh (energy) and prefer to have only positive values
model = LinearRegression()

# Fit 
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
cod = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Coefficient of Determination (COD): {cod}')
print(f'R-squared: {r2}')

## Plot of all features against the Total target (wind, solar, hydro)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

features = ['value_Global_Horizontal_Irrandiance',
    'value_Relative_Humidty',
    'value_Temperature',
    'value_Total_Precipitation']

num_features = len(features)
num_rows = num_features // 2 + num_features % 2  # Adjust for an odd number of features

# Define colors for regression lines
line_colors = ['red', 'blue', 'green', 'purple']

# Plotting subplots for each feature
fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(15, num_rows * 5))
fig.tight_layout(pad=5.0)

for i, (feature, color) in enumerate(zip(features, line_colors)):
    row, col = divmod(i, 2)
    
    # Plotting scatter plot with regression line on log scale for y-axis
    sns.regplot(x=feature, y='total target (wind, solar, hydro)', data=final, ax=axes[row, col], scatter_kws={'alpha':0.5}, line_kws={'color': color})
    
    axes[row, col].set_title(f'{feature} vs Total Target')
    axes[row, col].set_xlabel(feature)
    axes[row, col].set_ylabel('Total Target Energy (GWh)')
    
    # Set y-axis to log scale
    axes[row, col].set_yscale('log')

# Remove empty subplots if there are an odd number of features
if num_features % 2 != 0:
    fig.delaxes(axes[-1, -1])

plt.show()


In [ ]:
## We first select the columns that are gonna be part of our X 
##columns_X = [col for (col in final.columns if 'value' in col)]or (col in final.columns if 'Country' in col)]
columns_X = [col for col in final.columns if 'value' in col or 'Country' in col or 'Month_year' in col]

##  We're selecting the columns named with 'value'
X = final[columns_X]
X

In [ ]:
#categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns

#one-hot encoding to categorical columns
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)
X

# Lasso Analysis

In [ ]:
from sklearn.linear_model import Lasso, Ridge

In [ ]:
X = final[columns_X]
X

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split

##  We're selecting the columns named with 'value'
X = final[columns_X] # For the Lasso we use all our features, even the ones that are highly correlated

# Drop the original Timestamp column if needed
X = X.drop(columns=['Month_year', 'Country'])
y = final['total target (wind, solar, hydro)']

# Assuming X_train, y_train are your training data and labels
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a range of values for alpha and max_iter
alpha_values = [0.001, 0.01, 0.1, 1, 10, 100]
max_iter_values = [1500, 2000, 2100, 2200, 2300, 2400, 2500]

# Create a parameter grid for GridSearchCV
param_grid = {'alpha': alpha_values, 'max_iter': max_iter_values}

# Create a Lasso regression model
lasso_model = Lasso()

# Use GridSearchCV to find the best alpha and max_iter
grid_search = GridSearchCV(lasso_model, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_alpha = grid_search.best_params_['alpha']
best_max_iter = grid_search.best_params_['max_iter']

# Evaluate the model on the validation set with the best hyperparameters
lasso_best = Lasso(alpha=best_alpha, max_iter=best_max_iter)
lasso_best.fit(X_train, y_train)
y_pred_val = lasso_best.predict(X_val)
mse_val = mean_squared_error(y_val, y_pred_val)

y_pred_lasso = lasso_best.predict(X_test)
cod_lasso = r2_score(y_test, y_pred_lasso)

print(f'Best alpha: {best_alpha}')
print(f'Best max_iter: {best_max_iter}')
print(f'Mean Squared Error on Validation Set: {mse_val}')
print(f'Best Coefficient of Determination (COD) - Lasso: {cod_lasso}')


In [ ]:
##  We're selecting the columns named with 'value'
X = final[columns_X] # For the Lasso we use all our features, even the ones that are highly correlated

# Drop the original Timestamp column if needed
X = X.drop(columns=['Month_year', 'Country'])
y = final['total target (wind, solar, hydro)']

# Split training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#  L1 regularization (Lasso)
model_lasso = Lasso(alpha=0.01, max_iter=2600) ## Play with different alpha
model_lasso.fit(X_train, y_train)

# predictions on the testing data
y_pred_lasso = model_lasso.predict(X_test)

# Mean Absolute Error
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)

# Coefficient of Determination (COD)
cod_lasso = r2_score(y_test, y_pred_lasso)

print(f'Mean Absolute Error (Lasso): {mae_lasso}')
print(f'Coefficient of Determination (COD) - Lasso: {cod_lasso}')

# Polynomial Regression

### We first look for the best polynomial degree of 'total target (wind, solar, hydro)'

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split

##  We're selecting the columns named with 'value'
X = final[columns_X] # For the Lasso we use all our features, even the ones that are highly correlated

# Drop the original Timestamp column if needed
X = X.drop(columns=['Month_year', 'Country'])
y = final['total target (wind, solar, hydro)']

# Assuming X, y are your full dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a range of values for polynomial degree and number of folds
degree_values = [1, 2, 3, 4, 5]  

# Create a parameter grid for GridSearchCV
param_grid = {'polynomialfeatures__degree': degree_values}

# Create a Polynomial Regression model using a pipeline
poly_model = make_pipeline(PolynomialFeatures(), LinearRegression())
poly_model


In [ ]:
# Use GridSearchCV to find the best polynomial degree
grid_search = GridSearchCV(poly_model, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)


In [ ]:
# Get the best polynomial degree
best_degree = grid_search.best_params_['polynomialfeatures__degree']

# Fit the Polynomial Regression model with the best degree on the full training set
poly_model_best = make_pipeline(PolynomialFeatures(degree=best_degree), LinearRegression())
poly_model_best.fit(X_train, y_train)

# Make predictions on the test set
y_pred_poly = poly_model_best.predict(X_test)

# Calculate R-squared (Coefficient of Determination)
cod_poly = r2_score(y_test, y_pred_poly)

# Print the results
print(f'Best polynomial degree: {best_degree}')
print(f'Mean Squared Error on Validation Set: {grid_search.best_score_}')  # Negative of Mean Squared Error
print(f'Coefficient of Determination (COD) for Polynomial Regression: {cod_poly}')

## We now look for the best polynomial degree of 'Solar'

In [ ]:
##  We're selecting the columns named with 'value'

X = final[columns_X] # For the Lasso we use all our features, even the ones that are highly correlated

# Drop the original Timestamp column if needed
X = X.drop(columns=['Month_year', 'Country'])
y_solar = final['Solar']

# Assuming X, y are your full dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_solar, test_size=0.2, random_state=42)

# Define a range of values for polynomial degree and number of folds
degree_values = [1, 2, 3, 4, 5]  

# Create a parameter grid for GridSearchCV
param_grid = {'polynomialfeatures__degree': degree_values}

# Create a Polynomial Regression model using a pipeline
poly_model = make_pipeline(PolynomialFeatures(), LinearRegression())

In [ ]:
# Use GridSearchCV to find the best polynomial degree
grid_search_solar = GridSearchCV(poly_model, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search_solar.fit(X_train, y_train)

In [ ]:
# Get the best polynomial degree
best_degree_solar = grid_search_solar.best_params_['polynomialfeatures__degree']

# Fit the Polynomial Regression model with the best degree on the full training set
poly_model_best_solar = make_pipeline(PolynomialFeatures(degree=best_degree_solar), LinearRegression())
poly_model_best_solar.fit(X_train, y_train)

# Make predictions on the test set
y_pred_poly_solar = poly_model_best_solar.predict(X_test)

# Calculate R-squared (Coefficient of Determination)
cod_poly_solar = r2_score(y_test, y_pred_poly_solar)

# Print the results
print(f'Best polynomial degree: {best_degree_solar}')
print(f'Mean Squared Error on Validation Set: {grid_search_solar.best_score_}')  # Negative of Mean Squared Error
print(f'Coefficient of Determination (COD) for Polynomial Regression: {cod_poly_solar}')

## We now look for the best polynomial degree of 'Wind'

In [ ]:
##  We're selecting the columns named with 'value'

X = final[columns_X] # For the Lasso we use all our features, even the ones that are highly correlated

# Drop the original Timestamp column if needed
X = X.drop(columns=['Month_year', 'Country'])
y_wind = final['Wind']

# Assuming X, y are your full dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_wind, test_size=0.2, random_state=42)

# Define a range of values for polynomial degree and number of folds
degree_values = [1, 2, 3, 4, 5]  

# Create a parameter grid for GridSearchCV
param_grid = {'polynomialfeatures__degree': degree_values}

# Create a Polynomial Regression model using a pipeline
poly_model = make_pipeline(PolynomialFeatures(), LinearRegression())

In [ ]:
# Use GridSearchCV to find the best polynomial degree
grid_search_wind = GridSearchCV(poly_model, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search_wind.fit(X_train, y_train)

In [ ]:
# Get the best polynomial degree
best_degree_wind = 3
##grid_search_wind.best_params_['polynomialfeatures__degree']

# Fit the Polynomial Regression model with the best degree on the full training set
poly_model_best_wind = make_pipeline(PolynomialFeatures(degree=best_degree_wind), LinearRegression())
poly_model_best_wind.fit(X_train, y_train)

# Make predictions on the test set
y_pred_poly_wind = poly_model_best_wind.predict(X_test)

# Calculate R-squared (Coefficient of Determination)
cod_poly_wind = r2_score(y_test, y_pred_poly_wind)

# Print the results
print(f'Best polynomial degree: {best_degree_wind}')
print(f'Mean Squared Error on Validation Set: {grid_search_wind.best_score_}')  # Negative of Mean Squared Error
print(f'Coefficient of Determination (COD) for Polynomial Regression: {cod_poly_wind}')

## We now look for the best polynomial degree of 'Hydro'

In [ ]:
##  We're selecting the columns named with 'value'

X = final[columns_X] # For the Lasso we use all our features, even the ones that are highly correlated

# Drop the original Timestamp column if needed
X = X.drop(columns=['Month_year', 'Country'])
y_hydro = final['Hydro']

# Assuming X, y are your full dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_hydro, test_size=0.2, random_state=42)

# Define a range of values for polynomial degree and number of folds
degree_values = [1, 2, 3, 4, 5]  

# Create a parameter grid for GridSearchCV
param_grid = {'polynomialfeatures__degree': degree_values}

# Create a Polynomial Regression model using a pipeline
poly_model = make_pipeline(PolynomialFeatures(), LinearRegression())

In [ ]:
# Use GridSearchCV to find the best polynomial degree
grid_search_hydro = GridSearchCV(poly_model, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search_hydro.fit(X_train, y_train)

In [ ]:
# Get the best polynomial degree
best_degree_hydro = grid_search_hydro.best_params_['polynomialfeatures__degree']

# Fit the Polynomial Regression model with the best degree on the full training set
poly_model_best_hydro = make_pipeline(PolynomialFeatures(degree=best_degree_hydro), LinearRegression())
poly_model_best_hydro.fit(X_train, y_train)

# Make predictions on the test set
y_pred_poly_hydro = poly_model_best_hydro.predict(X_test)

# Calculate R-squared (Coefficient of Determination)
cod_poly_hydro = r2_score(y_test, y_pred_poly_hydro)

# Print the results
print(f'Best polynomial degree: {best_degree_hydro}')
print(f'Mean Squared Error on Validation Set: {grid_search_hydro.best_score_}')  # Negative of Mean Squared Error
print(f'Coefficient of Determination (COD) for Polynomial Regression: {cod_poly_hydro}')

# XGBoost Analysis

In [ ]:
#categorical columns
categorical_columns = final.select_dtypes(include=['object']).columns

#one-hot encoding to categorical columns
X = pd.get_dummies(final, columns=categorical_columns, drop_first=True)

In [ ]:
X = X.drop(columns=['Month_year'])

In [ ]:
import xgboost as xgb

##X = pd.get_dummies(final, columns=categorical_columns, drop_first=True)

y = final['total target (wind, solar, hydro)']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Fit 
model.fit(X_train, y_train)

# predict testing data
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
cod = r2_score(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)


print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Coefficient of Determination (COD): {cod}')
print(f'R-squared: {r2}')

### We are now running the model having taken China out of the dataset 

In [ ]:
#categorical columns
categorical_columns = final.select_dtypes(include=['object']).columns

#one-hot encoding to categorical columns
X = pd.get_dummies(final, columns=categorical_columns, drop_first=True)
X = X.drop(columns=['Month_year'])
X

In [ ]:
X_no_China = X.drop([column for column in X.columns if 'china' in column.lower()], axis=1)

In [ ]:
import xgboost as xgb

##X = pd.get_dummies(final, columns=categorical_columns, drop_first=True)

y = final['total target (wind, solar, hydro)']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_no_China, y, test_size=0.2, random_state=42)

# XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Fit 
model.fit(X_train, y_train)

# predict testing data
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
cod = r2_score(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)


print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Coefficient of Determination (COD): {cod}')
print(f'R-squared: {r2}')

## XG Boost for the Solar energy

In [ ]:
import xgboost as xgb

##X = pd.get_dummies(final, columns=categorical_columns, drop_first=True)

X
y = final['Solar']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Fit 
model.fit(X_train, y_train)

# predict testing data
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
cod = r2_score(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)


print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Coefficient of Determination (COD): {cod}')
print(f'R-squared: {r2}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

X
y = final['Solar']


# Function to plot learning curves
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

# Plot learning curve
plot_learning_curve(model, "Learning Curve", X, y, cv=5, n_jobs=-1)

plt.show()


# Merging the preprocessing and the models

In [55]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import pandas as pd
import numpy as np

df = pd.read_csv('/Users/sylvainvanhuysse/code/VonRiecken/Power-Predict/power_predict/data/merged_dataset2023-12-01 18:08:35.062618.csv')
df.head(5)

,Unnamed: 0,Month_year,Country,Balance,Combustible_Renewables,Hydro,Other_Renewables,Solar,Total_Renewables__Hydro__Geo__Solar__Wind__Other_,Wind,value_CDD_18,value_CDD_21,value_Global_Horizontal_Irrandiance,value_HDD_16,value_HDD_18,value_Heat_index,value_Relative_Humidty,value_Temperature,value_Total_Precipitation,total_sol_wind_hyd
0,0,2010-04-01,Australia,Net Electricity Production,216.287,1044.406,0.00,26.811,1638.098,350.511,0.111464,3.712857e-02,0.598917,0.019489,0.038689,0.639927,0.399736,0.867825,0.068503,1421.728
1,1,2010-04-01,Austria,Net Electricity Production,350.383,2504.130,0.00,9.965,2995.696,131.107,0.000126,0.000000e+00,0.611575,0.255355,0.300292,0.435803,0.605880,0.556986,0.131309,2645.202
2,2,2010-04-01,Belgium,Net Electricity Production,383.177,141.898,0.00,80.419,702.509,97.015,0.002296,0.000000e+00,0.600754,0.218429,0.264712,0.459624,0.573670,0.605589,0.049963,319.332
3,3,2010-04-01,Canada,Net Electricity Production,712.988,28243.738,2.44,24.184,29709.301,725.951,0.000159,6.825537e-08,0.564642,0.275212,0.319491,0.423290,0.622398,0.397319,0.068166,28993.873
4,4,2010-04-01,Chile,Net Electricity Production,152.826,1740.416,0.00,0.000,1913.601,20.359,0.003268,1.182407e-04,0.479870,0.126060,0.175605,0.518065,0.551867,0.612198,0.197151,1760.775


In [56]:
# --- Data Preprocessing ---
# Setting Country + Month year as Index
df['Country_Month'] = df['Country'] + '_' + df['Month_year'].astype(str)
df = df.set_index('Country_Month')
df

,Unnamed: 0,Month_year,Country,Balance,Combustible_Renewables,Hydro,Other_Renewables,Solar,Total_Renewables__Hydro__Geo__Solar__Wind__Other_,Wind,value_CDD_18,value_CDD_21,value_Global_Horizontal_Irrandiance,value_HDD_16,value_HDD_18,value_Heat_index,value_Relative_Humidty,value_Temperature,value_Total_Precipitation,total_sol_wind_hyd
Country_Month,,,,,,,,,,,,,,,,,,,,
Australia_2010-04-01,0,2010-04-01,Australia,Net Electricity Production,216.2870,1044.4060,0.0000,26.8110,1638.0980,350.5110,0.111464,3.712857e-02,0.598917,0.019489,0.038689,0.639927,0.399736,0.867825,0.068503,1421.7280
Austria_2010-04-01,1,2010-04-01,Austria,Net Electricity Production,350.3830,2504.1300,0.0000,9.9650,2995.6960,131.1070,0.000126,0.000000e+00,0.611575,0.255355,0.300292,0.435803,0.605880,0.556986,0.131309,2645.2020
Belgium_2010-04-01,2,2010-04-01,Belgium,Net Electricity Production,383.1770,141.8980,0.0000,80.4190,702.5090,97.0150,0.002296,0.000000e+00,0.600754,0.218429,0.264712,0.459624,0.573670,0.605589,0.049963,319.3320
Canada_2010-04-01,3,2010-04-01,Canada,Net Electricity Production,712.9880,28243.7380,2.4400,24.1840,29709.3010,725.9510,0.000159,6.825537e-08,0.564642,0.275212,0.319491,0.423290,0.622398,0.397319,0.068166,28993.8730
Chile_2010-04-01,4,2010-04-01,Chile,Net Electricity Production,152.8260,1740.4160,0.0000,0.0000,1913.6010,20.3590,0.003268,1.182407e-04,0.479870,0.126060,0.175605,0.518065,0.551867,0.612198,0.197151,1760.7750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Spain_2022-09-01,6713,2022-09-01,Spain,Net Electricity Production,478.4796,1459.6225,1.5000,3213.8391,9351.9487,4198.5075,0.261232,1.424242e-01,0.622150,0.006335,0.014242,0.700143,0.447142,0.810801,0.086444,8871.9691
Sweden_2022-09-01,6714,2022-09-01,Sweden,Net Electricity Production,819.7182,5992.2339,0.0000,101.3747,8900.2441,1986.9173,0.000000,0.000000e+00,0.259719,0.152834,0.205551,0.497448,0.779980,0.603833,0.115480,8080.5259
Switzerland_2022-09-01,6715,2022-09-01,Switzerland,Net Electricity Production,173.8267,2753.3512,0.0000,337.7122,3276.0098,11.1196,0.009257,0.000000e+00,0.468806,0.122044,0.164441,0.527046,0.762141,0.624622,0.309720,3102.1830


In [57]:
# Separating features and target variables
X = df.drop(['Unnamed: 0', 'Month_year', 'Balance',
             'Combustible_Renewables', 'Hydro', 'Other_Renewables', 'Solar',
             'Total_Renewables__Hydro__Geo__Solar__Wind__Other_', 'Wind',
             'total_sol_wind_hyd', 'value_CDD_18', 'value_CDD_21',
             'value_HDD_16', 'value_HDD_18', 'value_Heat_index',], axis=1)
X

,Country,value_Global_Horizontal_Irrandiance,value_Relative_Humidty,value_Temperature,value_Total_Precipitation
Country_Month,,,,,
Australia_2010-04-01,Australia,0.598917,0.399736,0.867825,0.068503
Austria_2010-04-01,Austria,0.611575,0.605880,0.556986,0.131309
Belgium_2010-04-01,Belgium,0.600754,0.573670,0.605589,0.049963
Canada_2010-04-01,Canada,0.564642,0.622398,0.397319,0.068166
Chile_2010-04-01,Chile,0.479870,0.551867,0.612198,0.197151
...,...,...,...,...,...
Spain_2022-09-01,Spain,0.622150,0.447142,0.810801,0.086444
Sweden_2022-09-01,Sweden,0.259719,0.779980,0.603833,0.115480
Switzerland_2022-09-01,Switzerland,0.468806,0.762141,0.624622,0.309720


In [58]:
y = np.log1p(df[['Hydro', 'Solar', 'Wind', 'total_sol_wind_hyd']])
y

,Hydro,Solar,Wind,total_sol_wind_hyd
Country_Month,,,,
Australia_2010-04-01,6.952161,3.325432,5.862241,7.260331
Austria_2010-04-01,7.826096,2.394708,4.883612,7.880881
Belgium_2010-04-01,4.962131,4.399609,4.585121,5.769358
Canada_2010-04-01,10.248662,3.226209,6.588859,10.274874
Chile_2010-04-01,7.462454,0.000000,3.061473,7.474077
...,...,...,...,...
Spain_2022-09-01,7.286618,8.075533,8.342723,9.090765
Sweden_2022-09-01,8.698386,4.628640,7.594843,8.997336
Switzerland_2022-09-01,7.920937,5.825151,2.494824,8.040184


In [59]:
# Init list of numerical columns
num_features = X.select_dtypes(include=[np.number]).columns.tolist()
# Preprocessing pipeline
preprocessing_pipeline = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('scaler', MinMaxScaler())
        ]), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Country'])
    ])
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [60]:
X_train

,Country,value_Global_Horizontal_Irrandiance,value_Relative_Humidty,value_Temperature,value_Total_Precipitation
Country_Month,,,,,
Sweden_2016-01-01,Sweden,0.027044,0.867691,0.254947,0.075137
Colombia_2014-12-01,Colombia,0.590806,0.819458,0.890861,0.361444
Bulgaria_2017-10-01,Bulgaria,0.395385,0.625206,0.654267,0.146927
Norway_2012-10-01,Norway,0.114303,0.837463,0.472980,0.197401
Czech Republic_2022-10-01,Czech Republic,0.259456,0.840601,0.638973,0.049113
...,...,...,...,...,...
Slovenia_2019-06-01,Slovenia,0.833562,0.572844,0.837613,0.143178
Chile_2012-11-01,Chile,0.884209,0.486951,0.634063,0.186407
Czech Republic_2013-11-01,Czech Republic,0.115206,0.896267,0.509875,0.082996


In [64]:
list(X_train['Country'].unique())

['Sweden',
 'Colombia',
 'Bulgaria',
 'Norway',
 'Czech Republic',
 'Canada',
 'Slovak Republic',
 'Australia',
 'France',
 'Austria',
 'Hungary',
 'Iceland',
 'Croatia',
 'Germany',
 'United States',
 'Denmark',
 "People's Republic of China",
 'Argentina',
 'Greece',
 'Italy',
 'Korea',
 'Finland',
 'India',
 'Slovenia',
 'Ireland',
 'Brazil',
 'Belgium',
 'Chile',
 'Romania',
 'Estonia',
 'Mexico',
 'Lithuania',
 'Netherlands',
 'Portugal',
 'New Zealand',
 'Serbia',
 'Poland',
 'Malta',
 'Spain',
 'Luxembourg',
 'United Kingdom',
 'Latvia',
 'Switzerland',
 'Costa Rica',
 'Japan',
 'Cyprus']